In [1]:
using Pkg
Pkg.activate(".")

using BenchmarkTools

Activating environment at `~/Dev/ROOTIO/Project.toml`


In [2]:
using ROOTIO

┌ Info: Precompiling ROOTIO [3cd96dde-e98d-4713-81e9-a4a1b0235ce9]
└ @ Base loading.jl:1273


In [3]:
@ROOTIO.io struct Foo
    a::Int32
    b::Int32
    c::Float32
    d::Int32
end

In [4]:
Foo(1, 2, 3, 4)

Foo(1, 2, 3.0f0, 4)

In [5]:
data = rand(UInt8, 1000000000);

In [6]:
buf = IOBuffer(data);

In [7]:
ROOTIO.unpack(IOBuffer(rand(UInt8, 1000)), Foo)

UndefVarError: UndefVarError: types not defined

In [8]:
@btime ROOTIO.unpack($buf, Foo)

UndefVarError: UndefVarError: types not defined

In [44]:
@btime retrieve_big_endian_32_foo_hardcoded_using_ntoh($data)

  10.999 ns (1 allocation: 64 bytes)


Foo(543217594, 1640246230, 3.3872068f-33, 1564409259)

In [23]:
@btime IOBuffer(data)

  85.815 ns (1 allocation: 64 bytes)


IOBuffer(data=UInt8[...], readable=true, writable=false, seekable=true, append=false, size=16, maxsize=Inf, ptr=1, mark=-1)

In [7]:
data = Vector{UInt8}([0x00, 0x00, 0x00, 0x64, 0x00, 0x00, 0x00, 0x32, 0x00, 0x00, 0x00, 0x16, 0x00, 0x00, 0x00, 0x24]);

In [2]:
struct Foo
    a::Int32
    b::Int32
    c::Float32
    d::Int32
end

In [21]:
function retrieve_foo(data)
    buf = IOBuffer(data)
    Foo(read(buf, Int32), read(buf, Int32), read(buf, Float32), read(buf, Int32))
end

function retrieve_foo_via_ref(data)
    ref = Ref{Foo}()
    read!(IOBuffer(data), ref)
    return ref[]
end


function retrieve_big_endian_32_foo(data)
    ref = Ref{Foo}()
    @inbounds for idx in range(1; length=Int(length(data)/4), step=4)
        reverse!(data, idx, idx+3)
    end
    read!(IOBuffer(data), ref)
    return ref[]
end


function retrieve_big_endian_32_foo_hardcoded_using_reverse_and_ref(data)
    ref = Ref{Foo}()
    reverse!(data, 1, 4)
    reverse!(data, 5, 8)
    reverse!(data, 9, 12)
    read!(IOBuffer(data), ref)
    return ref[]
end


function retrieve_big_endian_32_foo_hardcoded_using_ntoh(data)
    buf = IOBuffer(data)
    Foo(ntoh(read(buf, Int32)), ntoh(read(buf, Int32)), ntoh(read(buf, Float32)), ntoh(read(buf, Int32)))
end

retrieve_big_endian_32_foo_hardcoded_using_ntoh (generic function with 1 method)

In [15]:
data = Vector{UInt8}([0x00, 0x00, 0x00, 0x64, 0x00, 0x00, 0x00, 0x32, 0x00, 0x00, 0x00, 0x16, 0x00, 0x00, 0x00, 0x24]);

In [19]:
@btime retrieve_foo(data)

UndefVarError: UndefVarError: retrieve_foo not defined

In [9]:
@btime retrieve_foo_via_ref($data)

  15.397 ns (2 allocations: 96 bytes)


Foo(1677721600, 838860800, 1.0339758f-25)

In [10]:
@btime retrieve_big_endian_32_foo($data)

  38.039 ns (2 allocations: 96 bytes)


Foo(100, 50, 3.1f-44)

In [11]:
@btime retrieve_big_endian_32_foo_hardcoded_using_reverse_and_ref($data)

  27.803 ns (2 allocations: 96 bytes)


Foo(100, 50, 3.1f-44)

In [12]:
@btime retrieve_big_endian_32_foo_hardcoded_using_ntoh($data)

  12.002 ns (1 allocation: 64 bytes)


Foo(100, 50, 3.1f-44)